### 목적
- LLM이 작업을 수행할 수 있도록 정보와 도구를 적절한 형식으로 제공하는 동적 시스템 구축

#### 컨텍스트 엔지니어링 vs 프롬프트 엔지니어링
| 구분    | 프롬프트 엔지니어링                 | 컨텍스트 엔지니어링                                        |
| ----- | -------------------------- | ------------------------------------------------- |
| 초점    | 한 번의 입력문 설계                | 전체 대화·상태·환경의 구조 설계                                |
| 단위    | 문자열 prompt                 | 구조화된 context object                               |
| 접근 방식 | “어떻게 물어볼까?”                | “무엇을 알고, 어떤 상황에서 대답해야 하나?”                        |
| 주요 기술 | few-shot, CoT, role prompt | context_schema, dynamic_prompt, middleware, state |
| 결과    | 즉시 품질 향상                   | 지속적 일관성, 개인화, 멀티턴 정확도                             |
| 비유    | 문장 기술                      | 시스템 설계                                            |


#### 컨텍스트 종류
| 컨텍스트 종류                | 목적                                      | 어디에 저장되나   | 지속 시간                |
| ---------------------- | --------------------------------------- | ---------- | -------------------- |
| **Model Context**      | 모델에게 주는 일시적 정보 (프롬프트, 메시지, 툴 목록 등)      | 요청 내부      | 🔹짧음 (한 번 호출용)       |
| **Tool Context**       | 툴 함수가 읽고 쓸 수 있는 환경(context/state/store) | Runtime 객체 | 🔸중간 (세션 내 유지)       |
| **Life-cycle Context** | 실행 전후 감시·제어용 (미들웨어, 요약, 검열 등)           | 미들웨어 계층    | 🔸지속 (여러 호출 간 유지 가능) |

- Model Context: 매 호출마다 새로 생성 (Transient)
- Tool/Life-cycle Context: 실행 중 혹은 장기적으로 유지 (Persistent)

#### 컨텍스트 데이터 구성
- context: 현재 사용자나 환경같은 고정 정보 -> 조회/참고만 하고 불변의 성질
    - ex) user_id, language, timezone
- state: 대화 중 잠깐 유지되는 단기 메모리 -> 진행 상황을 step마다 저장하고 갱신되는 성질
    - ex) 직전 메시지, 업로드 파일 요약
- store: 장기 메모리
    - ex) 사용자 선호, 과거 대화 요약, 영구 기록

#### 각 컨텍스트 별로 다루는 데이터 소스
| 컨트롤 계층                 | 읽는 데이터                      | 쓰는 데이터          | 주요 접근 방식                                                 | 비고          |
| ---------------------- | --------------------------- | --------------- | -------------------------------------------------------- | ----------- |
| **Model Context**      | `context`, `state`, `store` | ❌ (읽기 중심)       | `request.context`, `request.state`, `request.store`      | 모델 호출 직전/직후 |
| **Tool Context**       | `state`, `store`            | ✅ (Command로 갱신) | `runtime.state`, `runtime.store`                         | 툴 함수 내 실행   |
| **Life-cycle Context** | `state`, `store`            | ✅ (요약/기록)       | `before_model`, `after_model`, `SummarizationMiddleware` | 대화 관리/압축    |


#### Model Context 조작 기법 - System prompt
- @dynamic_prompt
    - 실행 시점에 state, runtime.context, runtime.store를 읽어 system prompt를 생성

In [1]:
# state 사용 예시 코드
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

@dynamic_prompt
def system_prompt_selector(request: ModelRequest) -> str:
    """
    현재 대화 길이에 따라 system prompt를 자동 변경한다.
    - 메시지가 많으면: 간결하게
    - 메시지가 적으면: 일반 모드
    """
    # request.state["messages"] 를 request.messages 로 줄여서 적을 수 있다. -> 최신 버전 권장은 축약 형태
    # message_count = len(request.messages)
    message_count = len(request.state["messages"])
    print(len(request.state["messages"]))
    base = "너는 유용한 어시스턴트이다."
    if message_count > 5:
        base += "\n대화가 길어졌으니 간결하게 답변해라."
    else:
        base += "\n상세하고 친절하게 설명해라."
    return base

agent = create_agent(
    model=llm,
    tools=[],  
    middleware=[system_prompt_selector]
)

conversation_short = {"messages": [{"role": "user", "content": "인공지능이 뭐야?"}]}
conversation_long = {"messages": [{"role": "user", "content": "인공지능이 뭐야?"} for i in range(7)]}

print("▶ 짧은 대화:")
result_short = agent.invoke(conversation_short)
print(result_short["messages"][-1].content, "\n")

print("▶ 긴 대화:")
result_long = agent.invoke(conversation_long)
print(result_long["messages"][-1].content)

▶ 짧은 대화:
1
인공지능(Artificial Intelligence, AI)은 컴퓨터 시스템이나 기계가 인간처럼 사고, 학습, 문제 해결, 의사결정 등의 일을 수행할 수 있도록 하는 기술을 말합니다. 인공지능은 여러 가지 하위 분야로 나눌 수 있으며, 주로 다음과 같은 영역을 포함합니다.

1. **기계 학습 (Machine Learning)**: 이 분야는 컴퓨터가 데이터를 통해 패턴을 인식하고, 경험을 통해 성능을 향상시키는 방법에 중점을 둡니다. 기계 학습의 한 형태인 심층 학습(Deep Learning)은 인공 신경망을 사용하여 더 복잡한 문제를 해결하는 데 사용됩니다.

2. **자연어 처리 (Natural Language Processing, NLP)**: 이는 컴퓨터와 인간 언어 간의 상호작용을 다루는 분야로, 예를 들어 텍스트의 의미를 이해하고 생성하거나, 음성을 인식하는 기술이 포함됩니다.

3. **컴퓨터 비전 (Computer Vision)**: 이 분야는 컴퓨터가 이미지를 이해하고 해석하는 방법에 주목합니다. 얼굴 인식, 물체 탐지, 이미지 분류 등의 기술이 여기에 해당합니다.

4. **로보틱스 (Robotics)**: 로봇을 설계하고 제어하는 분야로, 인공지능을 활용하여 로봇이 환경을 인식하고, 작업을 수행하며, 자율적으로 움직이는 능력을 갖추게 합니다.

5. **전문 시스템 (Expert Systems)**: 특정 분야에서 전문가의 결정을 모방하는 시스템으로, 의료 진단, 금융 분석, 기술 지원 등에서 활용됩니다.

인공지능은 이미 여러 분야에서 널리 사용되고 있으며, 고객 서비스, 자율주행차, 추천 시스템, 의료 진단 등의 다양한 응용 분야에 적용되고 있습니다. 하지만 인공지능의 발전과 사용에는 윤리적 문제와 함께 사회적 영향도 있기 때문에, 이러한 측면도 함께 고민해야 합니다. 

▶ 긴 대화:
7
인공지능(Artificial Intelligence, AI)은 인간의 지능을 모방하거나 기능하는 컴퓨터 시스템이나 프로그램을 말

In [2]:
# context와 store 사용 예시 코드

import os
from dataclasses import dataclass
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from langgraph.store.memory import InMemoryStore

load_dotenv()

@dataclass
class Context:
    user_id: str

# 장기 스토어 준비
store = InMemoryStore()
store.put(("preferences",), "u-001", {"communication_style": "반말"})

# 동적 시스템 프롬프트: context.user_id로 store 조회
@dynamic_prompt
def store_aware_prompt(request: ModelRequest) -> str:
    user_id = request.runtime.context.user_id # 런타임에서 context로 들어온 user_id
    print("user id:", user_id)
    item = request.runtime.store.get(("preferences",), user_id) # 장기 메모리에서 user_id로 선호 검색
    base = "너는 유용한 조수이다."
    if item:
        style = item.value.get("communication_style", "균형잡힌 어투")
        base += f"\n유저의 선호도는 {style} 이다."
    return base

agent = create_agent(
    model=llm,
    tools=[],
    middleware=[store_aware_prompt],
    context_schema=Context,
    store=store,
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "간단히 요약해서 인공지능 정의해줘"}]},
    context=Context(user_id="u-001"),
)

print(result["messages"][-1].content)

user id: u-001
인공지능은 사람들이 하는 인지적인 작업을 컴퓨터가 수행할 수 있도록 만드는 기술이야. 여기에는 학습, 문제 해결, 추론, 언어 이해 등이 포함돼. 쉽게 말하면, 기계가 사람처럼 생각하고 행동하도록 만드는 거야.


#### Model Context 조작 기법 - Messages
- @wrap_model_call
    - 모델에 실제로 들어가는 메시지 리스트를 조작 -> 모델 입력을 직전에 다듬어 정확도, 안전성, 토큰 효율을 높인다.(요약, 정리, 필터링, 컨텍스트 주입 등)

In [3]:
# state 사용 예시 코드
import os
from typing import Callable, List, Dict, Any

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.agents import AgentState

load_dotenv()

class MyState(AgentState):
    uploaded_files: List[Dict[str, Any]]

@wrap_model_call
def inject_file_context(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    """이 세션에서 사용자가 업로드한 파일에 대한 컨텍스트를 삽입합니다."""
    uploaded_files_state: List[Dict[str, Any]] = request.state.get('uploaded_files', [])
    
    if uploaded_files_state:
        file_description = [
            f"{f['name']} ({f['type']}): {f['summary']}"
            for f in uploaded_files_state
        ]

        file_context = (
            "이 대화에서 사용 가능한 파일:\n"
            + "\n".join(file_description)
            + "\n답변할 때 이것을 참고 자료로 활용하세요."
        )

        augmented_messages = [
            {'role': 'system', 'content': file_context},
            *request.messages # 스프레드(unpack) 문법 -> request.messages을 풀어서 뒤에 이어붙임
        ]

        request = request.override(messages=augmented_messages) # 수정한 messages를 교체한다.
        # override()를 써야하는 이유?
        # ModelRequest 객체는 불변(immutable)이라서 request.messages=... 로 수정할 수 없다.
    
    return handler(request)

uploaded_files_state = [
    {"name": "project_plan.pdf", "type": "pdf", "summary": "Q4 마일스톤/데드라인 요약"},
    {"name": "error_log.txt", "type": "text", "summary": "최근 에러와 타임스탬프"},
]

agent = create_agent(
    model=llm,
    tools=[],
    middleware=[inject_file_context],
    state_schema=MyState
)

result = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "최근 로그를 참고해서 서비스 장애 원인 요약해줘."}
        ],
        "uploaded_files": uploaded_files_state
    },
)

print(result["messages"][-1].content)

최근 에러 로그를 분석한 결과, 서비스 장애의 원인은 다음과 같습니다:

1. **접속 지연**: 특정 시간대에 서버에 대한 요청이 급증하여 지연이 발생했습니다. 이는 서버의 리소스 초과 사용으로 이어졌습니다.

2. **네트워크 장애**: 로그에 네트워크 연결 오류가 기록되어 있어, 이로 인해 사용자의 요청이 완료되지 못했습니다.

3. **데이터베이스 오류**: 데이터베이스 접속 시도에서 시간 초과가 발생하여, 필요한 데이터를 불러오지 못하는 문제가 있었습니다.

4. **애플리케이션 버그**: 특정 기능에서 예외가 발생하고 로그에 캡처되어 오류가 반복적으로 발생했습니다. 이는 코드의 일부 로직 오류로 추측됩니다.

이러한 문제들은 시스템의 성능과 안정성에 영향을 미쳤으며, 즉각적인 해결책이 필요합니다. 각 원인을 개선하기 위한 조치가 요구됩니다.


In [4]:
# store와 context 사용 예시 코드
import os
from dataclasses import dataclass
from typing import Callable

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langgraph.store.memory import InMemoryStore

load_dotenv()

@dataclass
class Context:
    user_id: str

@wrap_model_call
def inject_writing_style(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    """Inject user's email writing style from Store."""
    user_id = request.runtime.context.user_id

    store = request.runtime.store
    writing_style = store.get(("writing_style",), user_id)

    if writing_style:
        style = writing_style.value
        style_context = f"""Your writing style:
        - Tone: {style.get('tone', 'professional')}
        - Typical greeting: "{style.get('greeting', 'Hi')}"
        - Typical sign-off: "{style.get('sign_off', 'Best')}"
        - Example email you've written:
        {style.get('example_email', '')}"""
        messages = [
            *request.messages,
            {"role": "user", "content": style_context}
        ]
        request = request.override(messages=messages)

    return handler(request)

llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

# 장기 메모리 준비
store = InMemoryStore()
store.put(("writing_style",), "u-001", {
    "tone": "friendly and concise",
    "greeting": "안녕하세요",
    "sign_off": "감사합니다",
    "example_email": "안녕하세요, 어제 요청주신 로그 분석 결과를 공유드립니다. 핵심 원인은 ...",
})

agent = create_agent(
    model=llm,
    tools=[],
    middleware=[inject_writing_style],
    context_schema=Context,
    store=store,
)

result = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "고객 A에게 장애 원인과 재발 방지 조치에 대한 이메일 초안 작성해줘."}
        ]
    },
    context=Context(user_id="u-001")
)

print(result["messages"][-1].content)


안녕하세요, 고객 A님.

최근 발생한 장애에 대한 원인과 재발 방지 조치를 말씀드리겠습니다.

장애의 주요 원인은 시스템의 데이터 처리 과정에서 예기치 않은 오류가 발생했기 때문입니다. 이 오류는 특정 조건에서 발생하는 문제로, 종합적인 분석을 통해 확인되었습니다.

재발 방지를 위해 다음과 같은 조치를 취할 예정입니다:
1. 오류를 수정한 코드 업데이트: 해당 오류를 수정한 패치를 곧 배포할 예정입니다.
2. 모니터링 강화: 시스템 모니터링을 강화하여 유사한 문제를 사전에 발견할 수 있도록 하겠습니다.
3. 주기적인 점검: 정기적으로 시스템 점검을 실시하여 안정성을 높이겠습니다.

추가적인 문의사항이나 불편사항이 있으시면 언제든지 말씀해 주세요.

감사합니다.


#### Model Context 조작 기법 - Tools
- 툴에는 명확한 이름, 설명, 인수 이름, 인수 설명이 필요하다. 
    - 단순히 메타데이터가 아니라 LLM이 도구를 사용하기 위한 모델의 추론을 안내하는 역할이다.
- 툴이 너무 많으면 모델에 과부하(컨텍스트 과부하)를 일으켜 오류가 증가하고, 도구가 너무 적으면 기능이 제한된다.
    - 사용 가능한 도구 세트를 조정하기 위해 Model Context(state, store, context)를 이용한다.
    - 예) 인증 전에는 public tool만 노출, 인증 후에는 private tool 활성화 등
- 예시 코드
    ```from langchain.tools import tool

    @tool(parse_docstring=True)
    def search_orders(
        user_id: str,
        status: str,
        limit: int = 10
    ) -> str:
        """Search for user orders by status.

        Use this when the user asks about order history or wants to check
        order status. Always filter by the provided status.

        Args:
            user_id: Unique identifier for the user
            status: Order status: 'pending', 'shipped', or 'delivered'
            limit: Maximum number of results to return
        """
        # Implementation here
        pass

In [5]:
# state 사용 예시 코드

from langchain.agents import create_agent, AgentState
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable
from langchain.tools import tool
from typing_extensions import TypedDict, NotRequired

@tool("public_search")
def public_search():
    """
    인증 없이 즉시 사용 가능한 기본 검색. 빠른 개요·일반 정보가 목표입니다.
    
    - 목적: 로그인 전/초기 단계에서 가벼운 정보 탐색을 신속히 제공합니다.
    - 전제조건: 없음(비인증 사용자 포함).
    - 사용 시점: 질문의 범위가 넓거나, 민감/정밀 정보가 필요 없을 때.
    - 출력: 요약 수준의 결과(개요/키워드/간단 참고).
    """
    print('public_search 호출')
    return

@tool("private_search")
def private_search():
    """
    인증 사용자 전용의 정밀 검색. 신뢰도·정확도·근거 제시를 우선합니다.
    
    - 목적: 개인화/권한이 필요한 정밀 탐색(예: 내부 지식, 고신뢰 출처 교차검증).
    - 전제조건: authenticated=True (미들웨어에서 보장).
    - 사용 시점: 구체적 기준/제한조건/정확한 근거가 필요한 요청일 때.
    - 출력: 출처·근거·필터 조건을 반영한 정밀 결과 및 다음 액션 제안.
    """
    print('private_search 호출')
    return

@tool("advanced_search")
def advanced_search():
    """
    고급 전략을 활용한 심층 검색. 멀티스텝/교차검증/재검색 루프로 완성도를 높입니다.
    
    - 목적: 난도 높은 질의에 대해 단계적 분해, 후보군 비교, 재시도·확장 탐색 수행.
    - 전제조건: authenticated=True 이면서 대화가 충분히 진행된 상태(미들웨어에서 조건화).
    - 사용 시점: 성능·정확도·재현성을 극대화해야 하거나, 대안 비교/랭킹이 필요할 때.
    - 출력: 비교표·스코어·리스크/트레이드오프 분석, 권장안 및 근거 묶음.
    """
    print('advanced_search 호출')
    return 

# state 스키마 정의
class Context(AgentState):
    authenticated: NotRequired[bool]

@wrap_model_call
def state_based_tools(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    """대화 상태를 기반으로 한 필터 도구입니다."""
    # state에서 사용자 인증 확인
    state = request.state
    is_authenticated = state.get('authenticated', False)
    message_count = len(state['messages'])

    # 인증된 상태에서만 특정 툴 적용
    if not is_authenticated:
        tools = [t for t in request.tools if t.name.startswith('public_')]
        request = request.override(tools=tools) # request.tools로 덮어 쓰지 못하기 때문에 override() 사용 
    elif message_count < 5: 
        # 대화 초기에는 툴 사용 제한
        tools = [t for t in request.tools if t.name != "advanced_search"]
        request = request.override(tools=tools)
    
    return handler(request)

agent = create_agent(
    model=llm,
    tools=[public_search, private_search, advanced_search],
    middleware=[state_based_tools],
    state_schema=Context
)

result = agent.invoke({
    'messages': [{'role': 'user', 'content': '인공지능에 대해 정밀 분석하고 검색해줘'}],
    "authenticated": False
})

result2 = agent.invoke({
    'messages': [{'role': 'user', 'content': '인공지능에 대해 정밀 분석하고 검색해줘'}],
    "authenticated": True
})

result3 = agent.invoke({
    'messages': [{'role': 'user', 'content': '인공지능에 대해 고급 검색해줘'} for t in range(6)],
    "authenticated": True
})


public_search 호출
private_search 호출
advanced_search 호출


In [6]:
# store와 context 사용 예제 코드
from dataclasses import dataclass
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable
from langgraph.store.memory import InMemoryStore

@dataclass
class Context:
    user_id: str

@tool
def search_tool():
    """일반 검색 툴"""
    print('search_tool 호출')
    return 

@tool
def analysis_tool():
    """자세한 검색 및 분석 툴"""
    print('analysis_tool 호출')
    return

@tool
def export_tool():
    """전문가용 검색 툴"""
    print('export_tool 호출')
    return

@wrap_model_call
def store_based_tools(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    """store의 선호도에 따른 필터"""
    user_id = request.runtime.context.user_id
    store = request.runtime.store

    feature_flags = store.get(('features',), user_id)

    if feature_flags:
        enabled_features = feature_flags.value.get('enabled_tools', [])
        print(f"[Middleware] user {user_id} enabled tools: {enabled_features}")
        tools = [t for t in request.tools if t.name in enabled_features]
        request = request.override(tools=tools)
    
    return handler(request)

# 임시로 선호 툴 메모리 넣어줌
store=InMemoryStore()
store.put(('features',), 'user001', {
    'enabled_tools': ['search_tool', 'analysis_tool', 'export_tool']
})

agent = create_agent(
    model=llm,
    tools=[search_tool, analysis_tool, export_tool],
    middleware=[store_based_tools],
    context_schema=Context,
    store=store
)

result = agent.invoke({
    'messages': [{'role': 'user', 'content': '인공지능에 대해 분석해줘'}]
    },
    context=Context(user_id='user001')
)

print(result.get('messages')[-1].content)

[Middleware] user user001 enabled tools: ['search_tool', 'analysis_tool', 'export_tool']
analysis_tool 호출
[Middleware] user user001 enabled tools: ['search_tool', 'analysis_tool', 'export_tool']
인공지능에 대한 구체적인 분석을 제공하기 위해 몇 가지 주요 주제를 다룰 수 있습니다:

1. **정의**: 인공지능(AI)은 인간의 인지 기능을 모방하는 컴퓨터 시스템이나 소프트웨어입니다. 이는 학습, 문제 해결, 언어 이해 및 비주얼 인식 등 다양한 작업을 수행할 수 있습니다.

2. **역사**: AI의 발전은 1950년대에 시작되어, 초창기에는 주로 규칙 기반 시스템에 의존했지만, 이후 머신러닝과 딥러닝 기술의 발전으로 인해 경이로운 진전을 이루었습니다.

3. **기술**: 인공지능의 주요 기술에는 머신러닝, 딥러닝, 자연어 처리(NLP), 컴퓨터 비전 등이 포함됩니다. 이들 기술은 데이터를 분석하고 예측하는 데 사용됩니다.

4. **응용 분야**: AI는 의료, 금융, 제조, 자율주행 자동차, 스마트 홈, 고객 서비스 등 다양한 분야에 활용되고 있습니다.

5. **윤리적 문제**: AI의 발전에 따라 개인정보 보호, 알고리즘의 편향성, 일자리 대체 등 다양한 윤리적 문제가 제기되고 있습니다.

6. **미래 전망**: AI는 앞으로도 계속 발전할 것이며, 인간 생활의 여러 측면을 변화시킬 것으로 기대되고 있습니다.

추가적으로 궁금한 점이나 구체적으로 알고 싶은 내용이 있다면 말씀해 주세요!


#### Model Context - model
- 모델마다 강점, 비용, 컨텍스트 윈도우가 다르다.
- 에이전트 실행 중에 task에 맞는 모델을 선정할 때 사용 가능하다.

In [7]:
# state 사용 예제 코드
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.chat_models import init_chat_model
from typing import Callable

large_model = init_chat_model("gpt-5")
standard_model = init_chat_model("gpt-4o")
efficient_model = init_chat_model("gpt-4o-mini")

@wrap_model_call
def state_based_model(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    """대화의 길이에 따라 적절한 모델 선택"""
    message_count = len(request.state["messages"])

    if message_count > 20:
        model = large_model
    elif message_count > 10:
        model = standard_model
    else:
        model = efficient_model

    request = request.override(model=model)
    print('현재 모델:', request.model.model_name)

    return handler(request)

agent = create_agent(
    model=llm,
    tools=[],
    middleware=[state_based_model]
)

result = agent.invoke({'messages': [{'role': 'user', 'content': '인공지능에 대해서 알려줘'}]})
result2 = agent.invoke({'messages': [{'role': 'user', 'content': '인공지능에 대해서 알려줘'} for t in range(21)]})
result3 = agent.invoke({'messages': [{'role': 'user', 'content': '인공지능에 대해서 알려줘'} for t in range(11)]})

현재 모델: gpt-4o-mini
현재 모델: gpt-5
현재 모델: gpt-4o


In [ ]:
# context와 store 사용 예제 코드
from dataclasses import dataclass
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.chat_models import init_chat_model
from typing import Callable
from langgraph.store.memory import InMemoryStore

@dataclass
class Context:
    user_id: str

MODEL_MAP = {
    "gpt-4o": init_chat_model("gpt-4o"),
    "gpt-4o-mini": init_chat_model("gpt-4o-mini"),
    "gpt-5": init_chat_model("gpt-5"),
}

@wrap_model_call
def store_based_model(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    """저장소에 저장된 선호도 기반으로 모델 선택"""
    user_id = request.runtime.context.user_id
    store = request.runtime.store
    user_prefs = store.get(('preferences',), user_id)

    if user_prefs:
        preferred_model = user_prefs.value.get('preferred_model')
        if preferred_model and preferred_model in MODEL_MAP:
            request = request.override(model=MODEL_MAP[preferred_model])

    print('현재 모델:', request.model.model_name)
    return handler(request) 
    
# 임시 메모리 저장
store = InMemoryStore()
store.put(('preferences',), 'user001', {'preferred_model': 'gpt-4o'})

agent = create_agent(
    model=llm,
    tools=[],
    middleware=[store_based_model],
    context_schema=Context,
    store=store
)

result = agent.invoke({
        'messages': [{'role': 'user', 'content': '인공지능에 대해서 알려줘'}]
    }, 
    context=Context(user_id='user001')
)

현재 모델: gpt-4o


#### Model Context - write
- 도구는 결과를 모델에 직접 반환할 뿐만 아니라 에이전트의 메모리를 업데이트할 수 있다.
    - 향후 단계에서 중요한 맥락을 활용할 수 있도록 한다.

In [37]:
# state 사용 예제 코드
from langchain.tools import tool, ToolRuntime
from langchain.agents import create_agent
from langgraph.types import Command
from langchain_core.messages import ToolMessage

@tool
def authenticate_user(
    password: str,
    runtime: ToolRuntime
) -> Command:
    """
    Authenticate user and update State.
    """
    if password == 'correct':
        return Command(
            update={
                "authenticated": True,
                "messages": [ToolMessage(
                    content="Authentication successful",
                    tool_call_id=runtime.tool_call_id
                )]
            }
        )
    else:
        return Command(
            update={
                "authenticated": False,
                "messages": [ToolMessage(
                    content="Authentication failed",
                    tool_call_id=runtime.tool_call_id
                )]
            }
        )
    
agent = create_agent(
    model=llm,
    tools=[authenticate_user]
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "비밀번호는 correct입니다"}]
})

print('[correct] :',result.get('messages')[-1].content)

result2 = agent.invoke({
    "messages": [{"role": "user", "content": "비밀번호는 wrong입니다"}]
})

print('[wrong] :', result2.get('messages')[-1].content)

[correct] : 인증이 성공적으로 완료되었습니다! 무엇을 도와드릴까요?
[wrong] : 비밀번호가 잘못되었습니다. 다시 시도해 주세요.


In [41]:
# context와 store 사용 예제 코드
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime
from langchain.agents import create_agent
from langgraph.store.memory import InMemoryStore

@dataclass
class Context:
    user_id: str

@tool
def save_preference(
    preference_key: str,
    preference_value: str,
    runtime: ToolRuntime[Context]
) -> str:
    """Save user preference to Store."""
    user_id = runtime.context.user_id

    # Read existing preferences
    store = runtime.store
    existing_prefs = store.get(("preferences",), user_id)

    # Merge with new preference
    prefs = existing_prefs.value if existing_prefs else {}
    prefs[preference_key] = preference_value

    # Write to Store: save updated preferences
    store.put(("preferences",), user_id, prefs)

    return f"Saved preference: {preference_key} = {preference_value}"

store = InMemoryStore()

agent = create_agent(
    model="gpt-4o",
    tools=[save_preference],
    context_schema=Context,
    store=store
)

# 에이전트 실행
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Save my theme preference as dark"}]},
    context=Context(user_id="user_123")
)

print(result.get('messages')[-1].content)
print(store.get(('preferences',), 'user_123').value)

Your theme preference has been saved as "dark."
{'theme': 'dark'}
